# FUNCTION

In [51]:
import pandas as pd
from io import StringIO
from google.cloud import storage
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold
)

def load_transcription_csv(file_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket('athena-nonprod-gcs')
    blob = bucket.blob(file_path)
    content = blob.download_as_text()

    return pd.read_csv(StringIO(content))[['speaker', 'transcript']]

def get_prompt(file_name, **kwargs):
    with open(f'./prompt/{file_name}', 'r') as file:
        prompt_template = file.read()

    return prompt_template.format(**kwargs)

def generate_prompt_formatted(layer: int, interaction_id: str, list_of_option):
    transcript_df = load_transcription_csv(f"athena-ws4a/athena-qapm/transcription/{interaction_id}/transcription_results.csv")
    transcript_df = transcript_df.where(pd.notnull(transcript_df), '')
    # json_dict = transcript_df.to_dict(orient='records')
    json_dict = {
        'speaker': transcript_df['speaker'].tolist(),
        'transcript': transcript_df['transcript'].tolist()
    }
    prompt = get_prompt(
        file_name=f'disposition_code_dirty_layer_{layer}.txt',
        transcript=json_dict,
        list_of_option=','.join(list_of_option),
    )
    return prompt

## MODEL LLM

In [52]:
model = GenerativeModel('gemini-1.5-flash-002')
generation_config = GenerationConfig(
    temperature=0,  # Example temperature value
    # top_k=1,         # Example top-k value
    # top_p=0.1        # Example top-p value
)

def generate_content(prompt):
    result = model.generate_content(
        contents=prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        },
        generation_config=generation_config
    )
    return result

# BATCH PROCESS

In [53]:
import json

with open('../../../dirty_data/action_to_product.json') as f:
    map_action_to_product = json.load(f)
    list_of_action = list(map_action_to_product.keys())

with open('../../../dirty_data/action_product_to_detail.json') as f:
    map_action_product_to_detail = json.load(f)

def get_layer_1(interaction_id: str, list_of_option):
    prompt = generate_prompt_formatted(1, interaction_id, list_of_option)
    response = generate_content(prompt).text
    disposition_result = json.loads(response.replace("```json", "").replace("```", "").strip())
    return disposition_result[0]['value']

def get_layer_2(layer_1: str, interaction_id: str):
    list_of_product = map_action_to_product[layer_1]
    prompt = generate_prompt_formatted(2, interaction_id, list_of_product)
    response = generate_content(prompt).text
    disposition_result = json.loads(response.replace("```json", "").replace("```", "").strip())
    return layer_1 + "|" + disposition_result[0]['value']

def get_layer_3(layer_2: str, interaction_id: str):
    list_of_conversational_context = map_action_product_to_detail[layer_2]
    prompt = generate_prompt_formatted(3, interaction_id, list_of_conversational_context)
    response = generate_content(prompt).text
    disposition_result = json.loads(response.replace("```json", "").replace("```", "").strip())
    return disposition_result

list_of_unknown_csv = [9151919416110000090, 9151919416110000091, 9151919416110000092]
output_data = []

for interaction_id in range(9151919416110000051, 9151919416110000099+1):
    if interaction_id in list_of_unknown_csv:
        continue
    try:
        layer_1 = get_layer_1(interaction_id, list_of_action)
        layer_2 = get_layer_2(layer_1, interaction_id)
        disposition_result = get_layer_3(layer_2, interaction_id)

        output_data.append({
            'interaction_id': interaction_id,
            'option_1': layer_2 + "|" + disposition_result[0]['value'],
            'explanation_1': disposition_result[0]['field3'],
            'option_2': layer_2 + "|" + disposition_result[1]['value'] if len(disposition_result) > 1 else None,
            'explanation_2': disposition_result[1]['field3'] if len(disposition_result) > 1 else None,
            'option_3': layer_2 + "|" +  disposition_result[2]['value'] if len(disposition_result) > 2 else None,
            'explanation_3': disposition_result[2]['field3'] if len(disposition_result) > 2 else None,
        })

        print(f"Processing interaction_id: {interaction_id}")
    except Exception as e:
        print(f"Error: {e} at interaction_id: {interaction_id}")

Processing interaction_id: 9151919416110000051
Processing interaction_id: 9151919416110000052
Processing interaction_id: 9151919416110000053
Processing interaction_id: 9151919416110000054
Processing interaction_id: 9151919416110000055
Processing interaction_id: 9151919416110000056
Processing interaction_id: 9151919416110000057
Processing interaction_id: 9151919416110000058
Processing interaction_id: 9151919416110000059
Processing interaction_id: 9151919416110000060
Processing interaction_id: 9151919416110000061
Processing interaction_id: 9151919416110000062
Processing interaction_id: 9151919416110000063
Processing interaction_id: 9151919416110000064
Processing interaction_id: 9151919416110000065
Processing interaction_id: 9151919416110000066
Processing interaction_id: 9151919416110000067
Processing interaction_id: 9151919416110000068
Processing interaction_id: 9151919416110000069
Processing interaction_id: 9151919416110000070
Processing interaction_id: 9151919416110000071
Processing in

In [54]:
df = pd.DataFrame(output_data)
df

,interaction_id,option_1,explanation_1,option_2,explanation_2,option_3,explanation_3
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah melaporkan tra...,Complain (General Customer)|Credit Card|CRD: K...,Penjelasan klasifikasi: Pengaduan terkait tra...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah mengajukan san...
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah melaporkan tra...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah meminta kartu ...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah mengalami kend...
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Nasabah mengalami kend...,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Masalah transaksi Auto...,Complain (General Customer)|Branchless|BLB: OC...,Penjelasan klasifikasi: Pelaporan kendala tran...
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah menanyakan tra...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Agen melakukan invest...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Proses investigasi tr...
4,9151919416110000055,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Nasabah mengalami kend...,Complain (General Customer)|Credit Card|CRD: C...,Penjelasan klasifikasi: Masalah utama adalah k...,Complain (General Customer)|Credit Card|CRD: B...,Penjelasan klasifikasi: Nasabah mempertanyakan...
5,9151919416110000056,Inquiry (General Customer)|Branchless|BLB: Kar...,Penjelasan klasifikasi: Nasabah menanyakan bia...,Inquiry (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Nasabah menanyakan bia...,Inquiry (General Customer)|Branchless|BLB: OCT...,Penjelasan klasifikasi: Nasabah menanyakan det...
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR St...,Penjelasan klasifikasi: Nasabah menanyakan sis...,Inquiry (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Nasabah ingin mengetah...,Inquiry (General Customer)|Lending|LND: KPR Ot...,Penjelasan klasifikasi: Agent tidak dapat mem...
7,9151919416110000058,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah menanyakan sis...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah menanyakan dam...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Percakapan berfokus p...
8,9151919416110000059,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah menanyakan sta...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah menanyakan sta...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Penjelasan klasifikasi: Nasabah meminta kenaik...
9,9151919416110000060,Report Status Checking|Credit Cards|CRD: Credi...,Penjelasan klasifikasi: Nasabah menanyakan sta...,None,None,None,None


# Combine data with ground truth

In [55]:
# read json file
import json

with open('./ground_truth/ground_truth.json') as f:
    ground_truth = json.load(f)

# map with ground truth
df['interaction_id'] = df['interaction_id'].astype(str)
df['ground_truth'] = df['interaction_id'].map(ground_truth)

# save to csv
df.to_csv('output.csv', index=False)

# Read the output.csv file

In [56]:
import pandas as pd
df = pd.read_csv('output.csv')
# check whether option_1 or option_2 or option_3 is equal to ground_truth, the result is True or False in "is_match" column
df['is_match'] = (df['ground_truth'] == df['option_1']) | (df['ground_truth'] == df['option_2']) | (df['ground_truth'] == df['option_3'])

df_view = df[['interaction_id', 'option_1', 'option_2', 'option_3', 'ground_truth', 'is_match']]
df_view

,interaction_id,option_1,option_2,option_3,ground_truth,is_match
0,9151919416110000051,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: K...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Branchless|BLB: AT...,False
1,9151919416110000052,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,True
2,9151919416110000053,Complain (General Customer)|Branchless|BLB: OC...,Complain (General Customer)|Branchless|BLB: OC...,Complain (General Customer)|Branchless|BLB: OC...,Complain (General Customer)|Branchless|BLB: OC...,False
3,9151919416110000054,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,True
4,9151919416110000055,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: C...,Complain (General Customer)|Credit Card|CRD: B...,Complain (General Customer)|Branchless|BLB: OC...,False
5,9151919416110000056,Inquiry (General Customer)|Branchless|BLB: Kar...,Inquiry (General Customer)|Branchless|BLB: OCT...,Inquiry (General Customer)|Branchless|BLB: OCT...,Inquiry (General Customer)|Funding (FND)|FND: ...,False
6,9151919416110000057,Inquiry (General Customer)|Lending|LND: KPR St...,Inquiry (General Customer)|Lending|LND: KPR Ot...,Inquiry (General Customer)|Lending|LND: KPR Ot...,Inquiry (General Customer)|Lending|LND: KPR Ot...,True
7,9151919416110000058,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Cr...,True
8,9151919416110000059,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Cr...,Inquiry (General Customer)|Credit Card|CRD: Pe...,False
9,9151919416110000060,Report Status Checking|Credit Cards|CRD: Credi...,NaN,NaN,Feedback/Report (General Customer)|Credit Card...,False


# CALCULATE ACCURACY

In [57]:
# calculate the mean of "is_match" column where ground_truth is not NaN
accuracy = df[df['ground_truth'].notnull()]['is_match'].mean()
accuracy*100

np.float64(37.5)

In [58]:
# GET WHICH LAYER GET THE WRONG PREDICTION
def split_option(df, column_name):
    """Memisahkan kolom berdasarkan '|' dan membuat kolom baru."""
    df[column_name + '_layer_1'] = df[column_name].str.split('|').str[0]
    df[column_name + '_layer_2'] = df[column_name].str.split('|').str[1]
    df[column_name + '_layer_3'] = df[column_name].str.split('|').str[2]
    return df

df = split_option(df, 'option_1')
df = split_option(df, 'option_2')
df = split_option(df, 'option_3')
df = split_option(df, 'ground_truth')

# check if layer 1, layer 2, layer 3 match with ground_truth layer 1, layer 2, layer 3
df['is_match_layer_1'] = (df['ground_truth_layer_1'] == df['option_1_layer_1'])
df['is_match_layer_2'] = (df['ground_truth_layer_2'] == df['option_1_layer_2'])
df['is_match_layer_3'] = (df['ground_truth_layer_3'] == df['option_1_layer_3']) | (df['ground_truth_layer_3'] == df['option_2_layer_3']) | (df['ground_truth_layer_3'] == df['option_3_layer_3'])

In [59]:
df_view = df[['interaction_id', 'option_1_layer_1', 'option_2_layer_1', 'option_3_layer_1', 'ground_truth_layer_1', 'is_match_layer_1']]
df_view

,interaction_id,option_1_layer_1,option_2_layer_1,option_3_layer_1,ground_truth_layer_1,is_match_layer_1
0,9151919416110000051,Complain (General Customer),Complain (General Customer),Complain (General Customer),Complain (General Customer),True
1,9151919416110000052,Complain (General Customer),Complain (General Customer),Complain (General Customer),Complain (General Customer),True
2,9151919416110000053,Complain (General Customer),Complain (General Customer),Complain (General Customer),Complain (General Customer),True
3,9151919416110000054,Complain (General Customer),Complain (General Customer),Complain (General Customer),Complain (General Customer),True
4,9151919416110000055,Complain (General Customer),Complain (General Customer),Complain (General Customer),Complain (General Customer),True
5,9151919416110000056,Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),True
6,9151919416110000057,Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),True
7,9151919416110000058,Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),True
8,9151919416110000059,Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),Inquiry (General Customer),True
9,9151919416110000060,Report Status Checking,NaN,NaN,Feedback/Report (General Customer),False


In [60]:
accuracy_layer_1 = df[df['ground_truth_layer_1'].notnull()]['is_match_layer_1'].mean()
print(f"Layer 1 accuracy: {accuracy_layer_1}")
# show df for layer 1
# df[['interaction_id', 'option_1_layer_1','option_2_layer_1','option_3_layer_1', 'ground_truth_layer_1', 'is_match_layer_1']]
df[['interaction_id', 'option_1_layer_1', 'ground_truth_layer_1', 'is_match_layer_1']]

Layer 1 accuracy: 0.825


,interaction_id,option_1_layer_1,ground_truth_layer_1,is_match_layer_1
0,9151919416110000051,Complain (General Customer),Complain (General Customer),True
1,9151919416110000052,Complain (General Customer),Complain (General Customer),True
2,9151919416110000053,Complain (General Customer),Complain (General Customer),True
3,9151919416110000054,Complain (General Customer),Complain (General Customer),True
4,9151919416110000055,Complain (General Customer),Complain (General Customer),True
5,9151919416110000056,Inquiry (General Customer),Inquiry (General Customer),True
6,9151919416110000057,Inquiry (General Customer),Inquiry (General Customer),True
7,9151919416110000058,Inquiry (General Customer),Inquiry (General Customer),True
8,9151919416110000059,Inquiry (General Customer),Inquiry (General Customer),True
9,9151919416110000060,Report Status Checking,Feedback/Report (General Customer),False


In [61]:
accuracy_layer_2 = df[df['ground_truth_layer_2'].notnull()]['is_match_layer_2'].mean()
print(f"Layer 2 accuracy: {accuracy_layer_2}")
df[['interaction_id','option_1_layer_2','option_2_layer_2','option_3_layer_2', 'ground_truth_layer_2', 'is_match_layer_2']]

Layer 2 accuracy: 0.675


,interaction_id,option_1_layer_2,option_2_layer_2,option_3_layer_2,ground_truth_layer_2,is_match_layer_2
0,9151919416110000051,Credit Card,Credit Card,Credit Card,Branchless,False
1,9151919416110000052,Credit Card,Credit Card,Credit Card,Credit Card,True
2,9151919416110000053,Branchless,Branchless,Branchless,Branchless,True
3,9151919416110000054,Credit Card,Credit Card,Credit Card,Credit Card,True
4,9151919416110000055,Credit Card,Credit Card,Credit Card,Branchless,False
5,9151919416110000056,Branchless,Branchless,Branchless,Funding (FND),False
6,9151919416110000057,Lending,Lending,Lending,Lending,True
7,9151919416110000058,Credit Card,Credit Card,Credit Card,Credit Card,True
8,9151919416110000059,Credit Card,Credit Card,Credit Card,Credit Card,True
9,9151919416110000060,Credit Cards,NaN,NaN,Credit Card,False


In [62]:
accuracy_layer_3 = df[df['ground_truth_layer_3'].notnull()]['is_match_layer_3'].mean()
print(f"Layer 3 accuracy: {accuracy_layer_3}")
df[['interaction_id','option_1_layer_3','option_2_layer_3','option_3_layer_3', 'ground_truth_layer_3', 'is_match_layer_3']]

Layer 3 accuracy: 0.525


,interaction_id,option_1_layer_3,option_2_layer_3,option_3_layer_3,ground_truth_layer_3,is_match_layer_3
0,9151919416110000051,CRD: Credit Cards Sanggahan Transaksi,CRD: Kendala Transaksi,CRD: Credit Cards Transaksi,BLB: ATM Gagal Tarik,False
1,9151919416110000052,CRD: Credit Cards Sanggahan Transaksi,CRD: Credit Cards Pergantian Kartu,CRD: Credit Cards Transaksi,CRD: Credit Cards Sanggahan Transaksi,True
2,9151919416110000053,BLB: OCTO Pay Kendala Transaksi,BLB: OCTO Pay Kendala Transaksi,BLB: OCTO Pay Others,BLB: OCTO Mobile Kendala Transaksi,False
3,9151919416110000054,CRD: Credit Cards Sanggahan Transaksi,CRD: Credit Cards Transaksi,CRD: Credit Cards Tagihan,CRD: Credit Cards Sanggahan Transaksi,True
4,9151919416110000055,CRD: Credit Cards Sanggahan Transaksi,CRD: Credit Cards Perubahan Data,CRD: Biaya/ Bunga/ Denda,BLB: OCTO Mobile Kendala Login,False
5,9151919416110000056,BLB: Kartu Debit Fitur,BLB: OCTO Mobile Fitur,BLB: OCTO Mobile Others,FND: Tabungan Others,False
6,9151919416110000057,LND: KPR Status,LND: KPR Others,LND: KPR Others,LND: KPR Others,True
7,9151919416110000058,CRD: Credit Cards Limit,CRD: Credit Cards Transaksi,CRD: Credit Cards Aktivasi dan PIN,CRD: Credit Cards Limit,True
8,9151919416110000059,CRD: Credit Cards Tagihan,CRD: Credit Cards Transaksi,CRD: Credit Cards Naik/Turun Limit Sementara,CRD: Pembayaran Tagihan Kartu Kredit,False
9,9151919416110000060,CRD: Credit Cards,NaN,NaN,CRD: Credit Cards,True
